<a href="https://colab.research.google.com/github/Khai2708/Ai_application/blob/main/Week13_Lab2_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This notebook was generated for TensorFlow 2.6.
#Processing words as a sequence: The sequence model approach
#A first practical example
#Downloading the data
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  9189k      0  0:00:08  0:00:08 --:--:-- 16.1M


In [ ]:
#Preparing the data
import os, pathlib, shutil, random 
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
  try:
    os.makedirs(val_dir / category)
  except FileExistsError:
    pass
  
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[- num_val_samples:]
  for fname in val_files:
    shutil.move(train_dir / category / fname,
                val_dir / category / fname)
    
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds= keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [ ]:
#Preparing integer sequence datasets
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens, output_mode="int", output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds= train_ds.map(
    lambda x, y: (text_vectorization(x), y), 
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y), 
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)


In [ ]:
#A sequence model built on one-hot encoded vector sequences
import tensorflow as tf
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout (0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model (inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy", 
              metrics= ["accuracy"])

model.summary ()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 20000)       0         
                                                                 
 bidirectional (Bidirectiona  (None, 64)               5128448   
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 5,128,513
Trainable params: 5,128,513
Non-trainable params: 0
___________________________________________________

In [ ]:
#Training a first basic sequence model
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
625/625 [==============================] - 178s 272ms/step - loss: 0.5311 - accuracy: 0.7419 - val_loss: 0.3694 - val_accuracy: 0.8574
Epoch 2/10
625/625 [==============================] - 171s 273ms/step - loss: 0.3263 - accuracy: 0.8801 - val_loss: 0.3026 - val_accuracy: 0.8844
Epoch 3/10
625/625 [==============================] - 170s 272ms/step - loss: 0.2581 - accuracy: 0.9096 - val_loss: 0.3705 - val_accuracy: 0.8798
Epoch 4/10
625/625 [==============================] - 171s 273ms/step - loss: 0.2222 - accuracy: 0.9229 - val_loss: 0.3357 - val_accuracy: 0.8792
Epoch 5/10
625/625 [==============================] - 171s 273ms/step - loss: 0.2013 - accuracy: 0.9323 - val_loss: 0.3247 - val_accuracy: 0.8812
Epoch 6/10
625/625 [==============================] - 169s 271ms/step - loss: 0.1748 - accuracy: 0.9431 - val_loss: 0.3521 - val_accuracy: 0.8858
Epoch 7/10
625/625 [==============================] - 171s 274ms/step - loss: 0.1500 - accuracy: 0.9521 - val_loss: 0.3503 -

In [ ]:
#Understanding word embeddings
#Learning word embeddings with the Embedding layer
#Instantiating an Embedding layer
embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256)


In [ ]:
#Model that uses an Embedding layer trained from scratch
inputs = keras.Input (shape=(None, ), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256) (inputs)
x = layers.Bidirectional (layers.LSTM(32))(embedded)
x = layers.Dropout (0.5)(x)
outputs = layers.Dense (1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy", 
              metrics= ["accuracy"])
model. summary ()

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                    save_best_only=True)
]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru.keras")
print(f"Test acc: {model.evaluate(int_test_ds) [1]:.3f}")


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               73984     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________

In [ ]:
#Understanding padding and masking
#Using an Embedding layer with masking enabled
inputs = keras.Input(shape= (None,), dtype="int64")
embedded = layers.Embedding(
input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model (inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()
callbacks = [
keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 64)               73984     
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________

In [ ]:
#Using pretrained word embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip


In [ ]:
#Parsing the GloVe word-embeddings file
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
  for line in f:
    word, coefs = line.split(maxsplit=1)
    coefs = np.fromstring(coefs, "f", sep=" ")
    embeddings_index[word] = coefs

print(f"Found {len(embeddings index)} word vectors.")

In [ ]:
#Preparing the GloVe word-embeddings matrix
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary ()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items ():
  if i < max tokens:
    embedding_vector = embeddings_index.get(word)
if embedding_vector is not None:
  embedding_matrix[i] = embedding_vector

In [ ]:

embedding layer = layers.Embedding(
    max_tokens, 
    embedding dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix), 
    trainable=False, 
    mask_zero=True,
)